## Safe Housing for Students

In [ ]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import requests
import json
import sys
import nltk
import spacy
import locationtagger
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

### Web Scraping/Retrieving DPS Alerts Data

In [2]:
links_df = pd.DataFrame()
links_df['ALERT LINK']=''
links_df

links = []

for i in range(1,50):
    #print("\n")
    alert_page_link = "https://dps.usc.edu/category/alerts/page/" + str(i) + "/"
    print("----------------", alert_page_link ,"----------------\n")
    
    URL = alert_page_link
    page = requests.get(URL)
    #print(page.text)
    soup = BeautifulSoup(page.content, "html.parser")
    #print(soup)
    
    results = soup.find(id="content")
    #print(results)

    job_elements = results.find_all("a", class_="read-more")
    #print(job_elements)    
    
    for link in job_elements:
        link_url = link["href"]
        #print(f"{link_url}\n")
        links.append(link_url)
    
links
links_df['ALERT LINK'] = links
links_df    

---------------- https://dps.usc.edu/category/alerts/page/1/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/2/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/3/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/4/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/5/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/6/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/7/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/8/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/9/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/10/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/11/ ----------------

---------------- https://dps.usc.edu/category/alerts/page/12/ ----------------

---------------- https://dps.usc.edu/category/ale

,ALERT LINK
0,https://dps.usc.edu/attempted-burglary/
1,https://dps.usc.edu/robbery-83/
2,https://dps.usc.edu/attempted-robbery-6/
3,https://dps.usc.edu/robbery-suspect-in-custody-4/
4,https://dps.usc.edu/robbery-82/
...,...
200,https://dps.usc.edu/robbery-kidnap-arrest/
201,https://dps.usc.edu/attempt-robbery-5/
202,https://dps.usc.edu/robbery-kidnapping/
203,https://dps.usc.edu/attempt-robbery-4/


In [3]:
links[47]

'https://dps.usc.edu/robbery-61/'

In [4]:
links_df.iloc[47][0]

'https://dps.usc.edu/robbery-61/'

In [5]:
cnt=0
ilist = []

for i in range(len(links_df)):
    URL = links_df['ALERT LINK'][i]     #https://dps.usc.edu/robbery-81/'
    if("usc-alert" in URL or "resolution" in URL or "advisory" in URL or "notification" in URL):
        cnt=cnt+1
        print(URL)
        ilist.append(i)
print(cnt)


for j in range(len(ilist)):
    links_df = links_df.drop([ilist[j]])

links_df = links_df.reset_index()
links_df = links_df.drop(['index'],  axis = 1)
links_df

https://dps.usc.edu/incident-resolution-attempted-robbery/
https://dps.usc.edu/university-safety-notification/
https://dps.usc.edu/community-safety-advisory-thefts-and-scams/
https://dps.usc.edu/community-safety-advisory-scams-3/
https://dps.usc.edu/community-safety-advisory-sexual-assault/
https://dps.usc.edu/community-safety-advisory-scams-2/
https://dps.usc.edu/usc-alert/
https://dps.usc.edu/community-advisory/
https://dps.usc.edu/community-safety-advisory-theft-by-trick/
https://dps.usc.edu/community-safety-advisory-scams/
https://dps.usc.edu/incident-resolution-burglary-arrest/
11


,ALERT LINK
0,https://dps.usc.edu/attempted-burglary/
1,https://dps.usc.edu/robbery-83/
2,https://dps.usc.edu/attempted-robbery-6/
3,https://dps.usc.edu/robbery-suspect-in-custody-4/
4,https://dps.usc.edu/robbery-82/
...,...
189,https://dps.usc.edu/robbery-arrest-17/
190,https://dps.usc.edu/robbery-kidnap-arrest/
191,https://dps.usc.edu/attempt-robbery-5/
192,https://dps.usc.edu/robbery-kidnapping/


In [6]:
link_df = links_df.copy()
link_df['INCIDENT TYPE']=''
link_df['INCIDENT DESCRIPTION']=''
link_df['DATE & TIME OF OCCURRENCE']=''
link_df['LOCATION']=''
#link_df['REPORT NUMBER']=''
link_df['VEHICLE DESCRIPTION']=''
link_df['SUSPECT DESCRIPTION']=''

link_df

,ALERT LINK,INCIDENT TYPE,INCIDENT DESCRIPTION,DATE & TIME OF OCCURRENCE,LOCATION,VEHICLE DESCRIPTION,SUSPECT DESCRIPTION
0,https://dps.usc.edu/attempted-burglary/,,,,,,
1,https://dps.usc.edu/robbery-83/,,,,,,
2,https://dps.usc.edu/attempted-robbery-6/,,,,,,
3,https://dps.usc.edu/robbery-suspect-in-custody-4/,,,,,,
4,https://dps.usc.edu/robbery-82/,,,,,,
...,...,...,...,...,...,...,...
189,https://dps.usc.edu/robbery-arrest-17/,,,,,,
190,https://dps.usc.edu/robbery-kidnap-arrest/,,,,,,
191,https://dps.usc.edu/attempt-robbery-5/,,,,,,
192,https://dps.usc.edu/robbery-kidnapping/,,,,,,


In [7]:
#Populating fields

#for j in range(2,2):
for j in range(len(link_df)):
    URL = link_df['ALERT LINK'][j]
    #URL = 'https://dps.usc.edu/burglary-suspect-in-custody-3/'
    
    print(j,"---"+URL+"---")   
    page = requests.get(URL)
    #print(page.text)

    soup = BeautifulSoup(page.content, "html.parser")
    soup
    soup.prettify()

    results = soup.find(id="primary")
    results

    #job_elements = results.find_all("div", class_="entry-content")
    #job_elements

    
    #************************************************************************************************
   
    
    title = results.find("h1", class_="entry-title")
    print("\nIncident Type : ",title.text)
    link_df['INCIDENT TYPE'].iloc[j] = title.text

    
    python_jobs1 = results.find_all("p")
    #rint("python_jobs1[0]", python_jobs1[0])
    #rint("python_jobs1[1]", python_jobs1[1])
    #rint("python_jobs1[2]", python_jobs1[2])
    #rint("python_jobs1[3]", python_jobs1[3])
    #lst=[]


    for i in range(len(python_jobs1)):
       
        if(":".lower() in python_jobs1[i].text.lower() and "Incident".lower() in python_jobs1[i].text.lower() and "Description".lower() in python_jobs1[i].text.lower()):
            if("Incident Description".lower() in python_jobs1[i].text.lower()):
                incident_description = python_jobs1[i].text.rpartition('Incident Description')
            if("Description of Incident".lower() in python_jobs1[i].text.lower()):
                incident_description = python_jobs1[i].text.rpartition('Description of Incident')
            key = incident_description[1]
            value = incident_description[2]
            #print(key, ":", value)
            value = value.replace(":", "", 1)
            value = value.strip()
            print(key, ":", value)
            link_df['INCIDENT DESCRIPTION'].iloc[j] = value
            continue

            
        if("Incident".lower() in python_jobs1[i].text.lower() and "Description".lower() in python_jobs1[i].text.lower()):
            if("Incident Description".lower() in python_jobs1[i].text.lower()):
                incident_description = python_jobs1[i].text.rpartition('Incident Description')
            if("Description of Incident".lower() in python_jobs1[i].text.lower()):
                incident_description = python_jobs1[i].text.rpartition('Description of Incident')
            key = incident_description[1]
            value = incident_description[2]
            #print(key, ":", value)
            value = value.replace(":", "", 1)
            value = value.strip()
            print(key, ":", value)
            link_df['INCIDENT DESCRIPTION'].iloc[j] = value
            continue
            

        if(":".lower() in python_jobs1[i].text.lower() and "DATE".lower() in python_jobs1[i].text.lower() and "TIME".lower() in python_jobs1[i].text.lower()):
            if("DATE & TIME OF OCCURRENCE".lower() in python_jobs1[i].text.lower()):
                date_time = python_jobs1[i].text.rpartition('DATE & TIME OF OCCURRENCE')
            if("Time & date of incident".lower() in python_jobs1[i].text.lower()):
                date_time = python_jobs1[i].text.rpartition('Time & date of incident')
            if("Date/Time".lower() in python_jobs1[i].text.lower()):
                date_time = python_jobs1[i].text.rpartition('Date/Time')
            key = date_time[1]
            value = date_time[2]
            #print(key, ":", value)
            value = value.replace(":", "", 1)
            value = value.strip()
            print(key, ":", value)
            link_df['DATE & TIME OF OCCURRENCE'].iloc[j] = value
            continue
            

        if(":".lower() in python_jobs1[i].text.lower() and "LOCATION".lower() in python_jobs1[i].text.lower()):
            if("LOCATION" in python_jobs1[i].text):
                location = python_jobs1[i].text.rpartition('LOCATION')
            if("Location" in python_jobs1[i].text):
                location = python_jobs1[i].text.rpartition('Location')
            key = location[1]
            value = location[2]
            #print(key, ":", value)
            value = value.replace(":", "", 1)
            value = value.strip()
            print(key, ":", value)
            if("Location" in value):
                value = value.replace("Location","")
            if("location" in value):
                value = value.replace("location","")
            link_df['LOCATION'].iloc[j] = value
            continue

            
#         if("REPORT NUMBER".lower() in python_jobs1[i].text.lower()):
#             incident_description = python_jobs1[i].text.rpartition('REPORT NUMBER')
#             key = incident_description[1]
#             value = incident_description[2]
#             #print(key, ":", value)
#             value = value.replace(":", "", 1)
#             value = value.strip()
#             print(key, ":", value)
#             link_df['REPORT NUMBER'].iloc[j] = value
#             continue
    
    
        if(":".lower() in python_jobs1[i].text.lower() and "VEHICLE".lower() in python_jobs1[i].text.lower()):
            if("VEHICLE DESCRIPTION".lower() in python_jobs1[i].text.lower()):
                vehicle = python_jobs1[i].text.rpartition('VEHICLE DESCRIPTION')
            if("SUSPECT VEHICLE".lower() in python_jobs1[i].text.lower()):
                vehicle = python_jobs1[i].text.rpartition('SUSPECT VEHICLE')
            key = vehicle[1]
            value = vehicle[2]
            #print(key, ":", value)
            value = value.replace(":", "", 1)
            value = value.strip()
            print(key, ":", value)
            link_df['VEHICLE DESCRIPTION'].iloc[j] = value
            continue
         
        
        if(":".lower() in python_jobs1[i].text.lower() and "SUSPECT DESCRIPTION".lower() in python_jobs1[i].text.lower()):
            suspect = python_jobs1[i].text.rpartition('SUSPECT DESCRIPTION')
            key = suspect[1]
            value = suspect[2]
            #print(key, ":", value)
            value = value.replace(":", "", 1)
            value = value.strip()
            print(key, ":", value)
            link_df['SUSPECT DESCRIPTION'].iloc[j] = value
            continue
    
            
    if(link_df['INCIDENT DESCRIPTION'][j])=='':
        python_jobs1 = results.find("p")
        link_df['INCIDENT DESCRIPTION'][j] = python_jobs1.text
    print("\n")    
            
    

0 ---https://dps.usc.edu/attempted-burglary/---

Incident Type :  Attempted Burglary
Incident Description : A student was awakened by the sound of someone trying to open his ground floor bedroom window. When the student approached the window, he was seen by the suspect who fled south on Severance Street.
 : Date & Time of Occurrence 04/15/2022 5:45 a.m.
Location : Off campus at the Pacific Apartments at 2637 Severance Street
 : Vehicle Description None
 : Suspect Description (provided by reporting party)


1 ---https://dps.usc.edu/robbery-83/---

Incident Type :  Robbery
Incident Description : Two students waiting for a ride service were approached by two suspects. One suspect displayed a handgun and demanded the students’ property. Both students surrendered their backpacks. Both suspects then walked away south on Figueroa Street.
 : Date & Time of Occurrence 04/12/2022 1:35 a.m.
Location : Off campus on the sidewalk in front of the Domino’s Pizza at 2803 S. Figueroa Street
 : Vehicle 


Incident Type :  Aggravated Assault
 : Incident description A USC student was walking eastbound on Exposition Boulevard near Bill Robertson Lane when an unhoused male approached her and struck her on the back of the head. The student continued walking eastbound on Exposition Boulevard and onto campus. The suspect then left the area for about 10 minutes. When he returned, he went to the corner of Exposition Boulevard and Vermont Avenue, where he assaulted a community member waiting at the bus stop. Other community members witnessed the assault and intervened.
DATE & TIME OF OCCURRENCE : 02/03/2022 8:30 a.m.
LOCATION : Exposition Boulevard and Vermont Avenue
VEHICLE DESCRIPTION : None
SUSPECT DESCRIPTION : (provided by reporting party)


13 ---https://dps.usc.edu/robbery-77/---

Incident Type :  Robbery
Incident Description : Two victims were robbed of their cellphones. The first victim was standing at the southeast corner of Jefferson Boulevard and Figueroa Street, in front of the Gale


Incident Type :  Robbery
Incident Description : Two suspects got out of a light blue Cadillac and robbed two men at gunpoint. Neither victim is affiliated with the university. The suspects were last seen driving southbound on Vermont Avenue.
DATE & TIME OF OCCURRENCE : 12/19/2021 3:07 p.m.
LOCATION : Off campus in the Smart and Final parking lot at 3607 S. Vermont Avenue
VEHICLE DESCRIPTION : A light blue Cadillac coupe, license plate number G70520.
SUSPECT DESCRIPTION : (provided by reporting party)


25 ---https://dps.usc.edu/robbery-68/---

Incident Type :  Robbery
Incident Description : While walking south on Orchard Avenue, a student was talking on his cell phone when he was approached by two suspects. The two suspects assaulted the student and took his cell phone. The suspects then got into a dark-colored car and drove away northbound on Orchard Avenue.
DATE & TIME OF OCCURRENCE : 12/10/2021 11:45 p.m.
LOCATION : Off-campus on the westside of Orchard Avenue, south of Adams Boule


Incident Type :  Robbery
Incident Description : The suspect drove alongside of the victim while she was walking northbound on Hoover. From inside the car, the suspect pointed a handgun at her and demanded her property. The victim, who is not affiliated with the university, gave the suspect her bookbag. The suspect then drove away southbound on Hoover.
DATE & TIME OF OCCURRENCE : 11/01/2021 8:05 a.m.
LOCATION : West sidewalk of Hoover Street north of Jefferson Boulevard
SUSPECT DESCRIPTION : (provided by reporting party)


38 ---https://dps.usc.edu/sexual-battery-fondling-11/---

Incident Type :  Sexual Battery/Fondling
 : Incident description The suspect was observed by Department of Public Safety (DPS) surveillance camera operators walking through a crowd of students who were leaving a social event. The suspect was seen on camera fondling at least two of the students. No individuals have contacted DPS to make a report.
Time & date of incident : 12:24 a.m. on Oct. 30, 2021
Location : 


Incident Type :  Grand Theft
 : Incident description The victim, who isn’t a student, advertised a Rolex watch for sale on an online marketplace. The victim agreed to drive to the location to meet the suspect. While the victim was seated in his car, the suspect reached through the open window, grabbed the watch, and ran to a waiting vehicle.
Time & date of incident : 5:20 p.m. on July 22, 2021
Location : Off campus in the 700 block of W. 27th Street
 : Suspect description The suspect is a Black male, approximately 20 to 30 years old, black hair, brown eyes, 5 feet 11 inches tall, 170 pounds with a tattoo on his right hand.
 : Suspect vehicle A 2015-2018 black Ford Mustang convertible


51 ---https://dps.usc.edu/robbery-59/---

Incident Type :  Robbery
 : Incident description The victim, who isn’t a student, advertised a Rolex watch on an online classifieds’ website. The suspect met the victim to make the purchase, displayed a handgun, and then stole the watch. The suspect then ran sou

...


Incident Type :  Robbery
 : Incident description One suspect approached the student, simulated a weapon and threatened her. A second suspect approached the student from behind and took the victim’s backpack which was on the ground next to her.
Time & date of incident : 6:30 p.m. on Nov. 15, 2020
Location : Off campus at the northeast corner of Normandie Avenue and 35th Place.
 : Suspect description Suspect number 1 is described as a male, with no further descriptive information.
 : Suspect vehicle No vehicle seen.
If you were in the vicinity during the time of this incident, the Department of Public Safety (DPS) asks for your help in identifying the suspect. If you have any information regarding this incident, please contact DPS at (213) 740-6000 or the Los Angeles Police Department (LAPD) at (213) 485-6571. If you are ever in immediate danger, call LAPD at 911 or DPS at (213) 740-4321.


102 ---https://dps.usc.edu/sexual-battery-fondling-8/---

Incident Type :  Sexual battery/fondlin


Incident Type :  Residential Burglary (Suspect in Custody)
Description of Incident : On July 30, 2020 at 3:30 a.m., the suspect entered the residence through an unlocked front door and removed a skateboard, gym bag and other property while the victims were sleeping. A few days later, after realizing they had been burglarized, the victims reviewed their security camera footage and observed the suspect entering and removing property.
Date/Time : August 7, 2020 at approximately 3 p.m.
Location : 23rd Street and Figueroa Street (off-campus)


117 ---https://dps.usc.edu/sexual-battery-10/---

Incident Type :  Sexual Battery
 : INCIDENT DESCRIPTION The victim was jogging on Ellendale Avenue approaching 29th Street when she observed the suspect walking south toward her. As she passed the suspect, he reached out and touched her buttocks. The suspect ran south on Ellendale Avenue.
DATE & TIME OF OCCURRENCE : August 5, 2020, 8:30 p.m.
LOCATION : Ellendale Avenue at 29th Street (off-campus)
SUSP


Incident Type :  Sexual Battery
 : INCIDENT DESCRIPTION As the female victim (unaffiliated with USC) was walking eastbound along the north side of 30th Street (north of Orchard Avenue), a male suspect riding a bicycle approached her from behind. The suspect grabbed the victim’s buttocks and continued riding eastbound on 30th Street toward McClintock Avenue.
DATE & TIME OF OCCURRENCE : Feb. 28, 2020 at approximately 7 p.m.
LOCATION : 1100 block of West 30th Street (off-campus)
SUSPECT DESCRIPTION : The suspect wore a dark hoodie.
SUSPECT VEHICLE : The suspect rode a dark bicycle.


131 ---https://dps.usc.edu/robbery-46/---

Incident Type :  Robbery
 : INCIDENT DESCRIPTION The male victim (USC staff member) watched the male suspect reach through the open window of his parked vehicle and remove several pieces of property. The victim pursued the suspect and tried to recover the stolen goods. The victim recovered some of his property, but the suspect ran southbound in the alley toward Jeff


Incident Type :  Grand Theft From Person
SUSPECT VEHICLE : Dark colored 4 door sedan
DATE & TIME OF OCCURRENCE : June 30, 2019 at 4:55 PM
LOCATION : McClintock Ave. and 30th St. (off campus)
SUSPECT DESCRIPTION : (provided by complainant)  The victim reported that the suspect had no distinguishing characteristics.
SUSPECT VEHICLE : Gray 2003-04 Mercedes SUV with dark tinted windows and a sun roof.


146 ---https://dps.usc.edu/robbery-arrest-24/---

Incident Type :  Robbery-Arrest


147 ---https://dps.usc.edu/robbery-41/---

Incident Type :  Robbery
DATE & TIME OF OCCURRENCE : May 19, 2019 at approximately 9:30 a.m.
LOCATION : 36th Place and Vermont Avenue (off campus)
SUSPECT DESCRIPTION : (provided by the victims)
SUSPECT VEHICLE : Silver 2005 Audi sedan, license number 7HYN526


148 ---https://dps.usc.edu/burglary-suspect-in-custody/---

Incident Type :  Burglary-Suspect in Custody


149 ---https://dps.usc.edu/sexual-battery-fondling-6/---

Incident Type :  Sexual Battery/Fondling
D


Incident Type :  Robbery at HSC
DATE & TIME OF OCCURRENCE : June 6, 2018 at 11:43 p.m.
LOCATION : 1500 Block of Soto Street, between Norfolk Street and Alcazar Street, L.A. (adjacent to Health Sciences Campus)
SUSPECT DESCRIPTION : Suspect 1: Male, Hispanic, approximately 28 to 30 years old, wearing grey tank top and light colored shorts.
SUSPECT VEHICLE : None seen


173 ---https://dps.usc.edu/robbery-auto-theft/---

Incident Type :  Robbery/Auto Theft
DATE & TIME OF OCCURRENCE : June 6, 2018, 3:40 PM
LOCATION : Shrine Parking Structure, 645 W. Jefferson Blvd.
SUSPECT DESCRIPTION : Female, Black, 25-30, 5-6”, 130 lbs., fair skin, reddish hair in a ponytail, tattoos on left arm, wearing a black jacket with shear sleeves, knee-high black boots, and light pants.  Victim described the suspect as being nicely dressed (see photo below).
SUSPECT VEHICLE : None seen


174 ---https://dps.usc.edu/robbery-suspects-in-custody/---

Incident Type :  Robbery Suspects In Custody
Location : Washingto

In [8]:
link_df.iloc[143]

ALERT LINK                      https://dps.usc.edu/sexual-battery-fondling-7/
INCIDENT TYPE                                          Sexual Battery/Fondling
INCIDENT DESCRIPTION                                       ALERT #: TW2019-007
DATE & TIME OF OCCURRENCE                              Oct. 4, 2019, 9:10 p.m.
LOCATION                                   3415 South Figueroa St. (on campus)
VEHICLE DESCRIPTION                                                           
SUSPECT DESCRIPTION          Male, Hispanic, dark complexion, age 25 to 29,...
Name: 143, dtype: object

In [9]:
#Populating empty fields

#for j in range(2,2):
for j in range(len(link_df)):
  
 for k in link_df.columns:
        
    if(link_df.iloc[j][k]==''):       
        #col = k

        URL = link_df['ALERT LINK'][j]
        #URL = 'https://dps.usc.edu/burglary-suspect-in-custody-3/'
        print(j,"---"+URL+"---")   
        page = requests.get(URL)
        #print(page.text)

        soup = BeautifulSoup(page.content, "html.parser")
        soup
        soup.prettify()

        results = soup.find(id="primary")
        results


        #************************************************************************************************


        title = results.find("h1", class_="entry-title")
        print("\nIncident Type : ",title.text)
        link_df['INCIDENT TYPE'].iloc[j] = title.text


        python_jobs1 = results.find_all("p")

        
        for i in range(len(python_jobs1)):
            if("Incident".lower() in python_jobs1[i].text.lower() and "Incident".lower() in k.lower()): 
                if(":".lower() in python_jobs1[i].text.lower() and "Incident".lower() in python_jobs1[i].text.lower() and "Description".lower() in python_jobs1[i].text.lower()):
                    if("Incident Description".lower() in python_jobs1[i].text.lower()):
                        incident_description = python_jobs1[i].text.rpartition('Incident Description')
                    if("Description of Incident".lower() in python_jobs1[i].text.lower()):
                        incident_description = python_jobs1[i].text.rpartition('Description of Incident')
                    key = incident_description[1]
                    value = incident_description[2]
                    #print(key, ":", value)
                    value = value.replace(":", "", 1)
                    value = value.strip()
                    #print(key, ":", value)
                    link_df['INCIDENT DESCRIPTION'].iloc[j] = value
                    break


            if("Incident".lower() in python_jobs1[i].text.lower() and "Incident".lower() in k.lower()): 
                if("Incident".lower() in python_jobs1[i].text.lower() and "Description".lower() in python_jobs1[i].text.lower()):
                    if("Incident Description".lower() in python_jobs1[i].text.lower()):
                        incident_description = python_jobs1[i].text.rpartition('Incident Description')
                    if("Description of Incident".lower() in python_jobs1[i].text.lower()):
                        incident_description = python_jobs1[i].text.rpartition('Description of Incident')
                    key = incident_description[1]
                    value = incident_description[2]
                    #print(key, ":", value)
                    value = value.replace(":", "", 1)
                    value = value.strip()
                    #print(key, ":", value)
                    link_df['INCIDENT DESCRIPTION'].iloc[j] = value
                    break


            if("Date".lower() in python_jobs1[i].text.lower() and "Date".lower() in k.lower()): 
                if(":".lower() in python_jobs1[i].text.lower() and "DATE".lower() in python_jobs1[i].text.lower() and "TIME".lower() in python_jobs1[i].text.lower()):
                    if("DATE & TIME OF OCCURRENCE".lower() in python_jobs1[i].text.lower()):
                        date_time = python_jobs1[i].text.rpartition('DATE & TIME OF OCCURRENCE')
                    if("Time & date of incident".lower() in python_jobs1[i].text.lower()):
                        date_time = python_jobs1[i].text.rpartition('Time & date of incident')
                    if("Date/Time".lower() in python_jobs1[i].text.lower()):
                        date_time = python_jobs1[i].text.rpartition('Date/Time')
                    key = date_time[1]
                    value = date_time[2]
                    #print(key, ":", value)
                    value = value.replace(":", "", 1)
                    value = value.strip()
                    #print(key, ":", value)
                    link_df['DATE & TIME OF OCCURRENCE'].iloc[j] = value
                    break


            if("Location".lower() in python_jobs1[i].text.lower() and "Location".lower() in k.lower()): 
                if(":".lower() in python_jobs1[i].text.lower() and "LOCATION".lower() in python_jobs1[i].text.lower()):
                    if("LOCATION" in python_jobs1[i].text):
                        location = python_jobs1[i].text.rpartition('LOCATION')
                    if("Location" in python_jobs1[i].text):
                        location = python_jobs1[i].text.rpartition('Location')
                    key = location[1]
                    value = location[2]
                    #print(key, ":", value)
                    value = value.replace(":", "", 1)
                    value = value.strip()
                    #print(key, ":", value)
                    if("Location" in value):
                        value = value.replace("Location","")
                    if("location" in value):
                        value = value.replace("location","")
                    link_df['LOCATION'].iloc[j] = value
                    break


            #         if("REPORT NUMBER".lower() in python_jobs1[i].text.lower()):
            #             incident_description = python_jobs1[i].text.rpartition('REPORT NUMBER')
            #             key = incident_description[1]
            #             value = incident_description[2]
            #             #print(key, ":", value)
            #             value = value.replace(":", "", 1)
            #             value = value.strip()
            #             print(key, ":", value)
            #             link_df['REPORT NUMBER'].iloc[j] = value
            #             continue


            if("Vehicle".lower() in python_jobs1[i].text.lower() and "Vehicle".lower() in k.lower()): 
                if(":".lower() in python_jobs1[i].text.lower() and "VEHICLE".lower() in python_jobs1[i].text.lower()):
                    if("VEHICLE DESCRIPTION".lower() in python_jobs1[i].text.lower()):
                        vehicle = python_jobs1[i].text.rpartition('VEHICLE DESCRIPTION')
                    if("SUSPECT VEHICLE".lower() in python_jobs1[i].text.lower()):
                        vehicle = python_jobs1[i].text.rpartition('SUSPECT VEHICLE')
                    key = vehicle[1]
                    value = vehicle[2]
                    #print(key, ":", value)
                    value = value.replace(":", "", 1)
                    value = value.strip()
                    #print(key, ":", value)
                    link_df['VEHICLE DESCRIPTION'].iloc[j] = value
                    break


            if("SUSPECT DESCRIPTION".lower() in python_jobs1[i].text.lower() and "SUSPECT DESCRIPTION".lower() in k.lower()): 
                if(":".lower() in python_jobs1[i].text.lower() and "SUSPECT DESCRIPTION".lower() in python_jobs1[i].text.lower()):
                    suspect = python_jobs1[i].text.rpartition('SUSPECT DESCRIPTION')
                    key = suspect[1]
                    value = suspect[2]
                    #print(key, ":", value)
                    value = value.replace(":", "", 1)
                    value = value.strip()
                    #print(key, ":", value)
                    link_df['SUSPECT DESCRIPTION'].iloc[j] = value
                    break


link_df

34 ---https://dps.usc.edu/robbery-64/---

Incident Type :  Robbery
35 ---https://dps.usc.edu/update-robbery/---

Incident Type :  Update: Robbery
36 ---https://dps.usc.edu/brandishing-a-firearm-3/---

Incident Type :  Brandishing a Firearm
37 ---https://dps.usc.edu/robbery-63/---

Incident Type :  Robbery
39 ---https://dps.usc.edu/sexual-battery-fondling-10/---

Incident Type :  Sexual Battery/Fondling
40 ---https://dps.usc.edu/sexual-assault-4/---

Incident Type :  Sexual Assault
40 ---https://dps.usc.edu/sexual-assault-4/---

Incident Type :  Sexual Assault
40 ---https://dps.usc.edu/sexual-assault-4/---

Incident Type :  Sexual Assault
45 ---https://dps.usc.edu/attempted-kidnapping/---

Incident Type :  Attempted Kidnapping
46 ---https://dps.usc.edu/burglary-suspect-in-custody-5/---

Incident Type :  Burglary-Suspect in Custody
46 ---https://dps.usc.edu/burglary-suspect-in-custody-5/---

Incident Type :  Burglary-Suspect in Custody
55 ---https://dps.usc.edu/robberies-suspect-in-custo


Incident Type :  Sexual Battery (Suspect in Custody)
133 ---https://dps.usc.edu/sexual-battery-suspect-in-custody-2/---

Incident Type :  Sexual Battery (Suspect in Custody)
136 ---https://dps.usc.edu/robbery-45/---

Incident Type :  Robbery
137 ---https://dps.usc.edu/robbery-44/---

Incident Type :  Robbery
139 ---https://dps.usc.edu/burglary-residential-5/---

Incident Type :  Burglary Residential
139 ---https://dps.usc.edu/burglary-residential-5/---

Incident Type :  Burglary Residential
139 ---https://dps.usc.edu/burglary-residential-5/---

Incident Type :  Burglary Residential
142 ---https://dps.usc.edu/sexual-battery-suspect-in-custody/---

Incident Type :  Sexual Battery (Suspect in Custody)
142 ---https://dps.usc.edu/sexual-battery-suspect-in-custody/---

Incident Type :  Sexual Battery (Suspect in Custody)
142 ---https://dps.usc.edu/sexual-battery-suspect-in-custody/---

Incident Type :  Sexual Battery (Suspect in Custody)
142 ---https://dps.usc.edu/sexual-battery-suspect-in-


Incident Type :  Attempt Robbery
193 ---https://dps.usc.edu/attempt-robbery-4/---

Incident Type :  Attempt Robbery
193 ---https://dps.usc.edu/attempt-robbery-4/---

Incident Type :  Attempt Robbery


,ALERT LINK,INCIDENT TYPE,INCIDENT DESCRIPTION,DATE & TIME OF OCCURRENCE,LOCATION,VEHICLE DESCRIPTION,SUSPECT DESCRIPTION
0,https://dps.usc.edu/attempted-burglary/,Attempted Burglary,A student was awakened by the sound of someone...,Date & Time of Occurrence 04/15/2022 5:45 a.m.,Off campus at the Pacific Apartments at 2637 S...,Vehicle Description None,Suspect Description (provided by reporting party)
1,https://dps.usc.edu/robbery-83/,Robbery,Two students waiting for a ride service were a...,Date & Time of Occurrence 04/12/2022 1:35 a.m.,Off campus on the sidewalk in front of the Dom...,Vehicle Description None,Suspect Description (provided by reporting party)
2,https://dps.usc.edu/attempted-robbery-6/,Attempted Robbery,The victim exited the Metro platform at Jeffer...,Date & Time of Occurrence 04/04/2022 4:19 p.m.,"Off-Campus, at the Metro train platform on Jef...",Vehicle Description None,Suspect Description (provided by reporting party)
3,https://dps.usc.edu/robbery-suspect-in-custody-4/,Robbery Suspect in Custody,The juvenile male suspect approached a USC stu...,Date & Time of Occurrence 03/30/2022 11:51 a.m...,"Off-Campus, on the northeast corner of Catalin...",Vehicle Description None,Suspect Description (provided by reporting par...
4,https://dps.usc.edu/robbery-82/,Robbery,"While at the ATM, the victim was approached by...",Date & Time of Occurrence 03/25/2022 3:41 a.m.,"Off-Campus, at the Bank of America located at ...",Vehicle Description Silver sedan with a sunroo...,Suspect Description (provided by reporting party)
...,...,...,...,...,...,...,...
189,https://dps.usc.edu/robbery-arrest-17/,Robbery Arrest,"At approximately 9:30 PM, a student was skateb...",,,DESCRIPTION None. Suspect left the area on foot.,
190,https://dps.usc.edu/robbery-kidnap-arrest/,Robbery/Kidnap Arrest,"While walking away from her residence, a stude...",,,,
191,https://dps.usc.edu/attempt-robbery-5/,Attempt Robbery,A female student and her mother were walking w...,"January 24, 2018 at approximately 2:20 P.M.",37th Street west of Vermont Avenue (off campus),Dark colored scooter similar to an URB-E,"(provided by the reporting parties) Male, Bla..."
192,https://dps.usc.edu/robbery-kidnapping/,Robbery/Kidnapping,As the victim (student) walked away from her r...,,,,


In [10]:
link_df.iloc[143]

ALERT LINK                      https://dps.usc.edu/sexual-battery-fondling-7/
INCIDENT TYPE                                          Sexual Battery/Fondling
INCIDENT DESCRIPTION                                       ALERT #: TW2019-007
DATE & TIME OF OCCURRENCE                              Oct. 4, 2019, 9:10 p.m.
LOCATION                                   3415 South Figueroa St. (on campus)
VEHICLE DESCRIPTION                                                           
SUSPECT DESCRIPTION          Male, Hispanic, dark complexion, age 25 to 29,...
Name: 143, dtype: object

In [11]:
# URL = 'https://dps.usc.edu/burglary-suspect-in-custody-3/'
# print("---"+URL+"---")   
# page = requests.get(URL)
# #print(page.text)

# soup = BeautifulSoup(page.content, "html.parser")
# soup
# soup.prettify()

# results = soup.find(id="primary")
# results

# #job_elements = results.find_all("div", class_="entry-content")
# #job_elements


# #************************************************************************************************


# title = results.find("h1", class_="entry-title")
# print("\nIncident Type : ",title.text)
# link_df['INCIDENT TYPE'].iloc[j] = title.text


# python_jobs1 = results.find_all("p")
# python_jobs1
# #lst=[]


In [12]:
data_index = [i for i in range(1,len(link_df)+1)]
data = link_df.copy()
data['INDEX']=data_index
#data.index = data_index
data.set_index('INDEX', inplace=True)
data.to_excel("alerts.xlsx", encoding='utf8')
data

,ALERT LINK,INCIDENT TYPE,INCIDENT DESCRIPTION,DATE & TIME OF OCCURRENCE,LOCATION,VEHICLE DESCRIPTION,SUSPECT DESCRIPTION
INDEX,,,,,,,
1,https://dps.usc.edu/attempted-burglary/,Attempted Burglary,A student was awakened by the sound of someone...,Date & Time of Occurrence 04/15/2022 5:45 a.m.,Off campus at the Pacific Apartments at 2637 S...,Vehicle Description None,Suspect Description (provided by reporting party)
2,https://dps.usc.edu/robbery-83/,Robbery,Two students waiting for a ride service were a...,Date & Time of Occurrence 04/12/2022 1:35 a.m.,Off campus on the sidewalk in front of the Dom...,Vehicle Description None,Suspect Description (provided by reporting party)
3,https://dps.usc.edu/attempted-robbery-6/,Attempted Robbery,The victim exited the Metro platform at Jeffer...,Date & Time of Occurrence 04/04/2022 4:19 p.m.,"Off-Campus, at the Metro train platform on Jef...",Vehicle Description None,Suspect Description (provided by reporting party)
4,https://dps.usc.edu/robbery-suspect-in-custody-4/,Robbery Suspect in Custody,The juvenile male suspect approached a USC stu...,Date & Time of Occurrence 03/30/2022 11:51 a.m...,"Off-Campus, on the northeast corner of Catalin...",Vehicle Description None,Suspect Description (provided by reporting par...
5,https://dps.usc.edu/robbery-82/,Robbery,"While at the ATM, the victim was approached by...",Date & Time of Occurrence 03/25/2022 3:41 a.m.,"Off-Campus, at the Bank of America located at ...",Vehicle Description Silver sedan with a sunroo...,Suspect Description (provided by reporting party)
...,...,...,...,...,...,...,...
190,https://dps.usc.edu/robbery-arrest-17/,Robbery Arrest,"At approximately 9:30 PM, a student was skateb...",,,DESCRIPTION None. Suspect left the area on foot.,
191,https://dps.usc.edu/robbery-kidnap-arrest/,Robbery/Kidnap Arrest,"While walking away from her residence, a stude...",,,,
192,https://dps.usc.edu/attempt-robbery-5/,Attempt Robbery,A female student and her mother were walking w...,"January 24, 2018 at approximately 2:20 P.M.",37th Street west of Vermont Avenue (off campus),Dark colored scooter similar to an URB-E,"(provided by the reporting parties) Male, Bla..."


In [13]:
# dict_data = data.to_dict()
# dict_data

# json_data = json.dumps(dict_data)
# json_data

# response = requests.put('https://dsci551-project-33b23-default-rtdb.firebaseio.com/.json',
# json_data)
# print(response)

### Locations - Frequency

In [14]:
# import nltk
# from nltk.tag import pos_tag
# from nltk.tag.stanford import StanfordNERTagger
# from nltk.chunk import tree2conlltags
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('treebank')
# nltk.download('averaged_perceptron_tagger')

In [15]:
remove_locs=['Boulevard','Smart','Southeast','Similar','West','Jesse','Incident','Public Safety','DPS','Avenue',
'Custody','Aggravated','Sexual','Texts','Date','IRN','South','Homeless','Original Incident','New','SUSPECT',
'Black','Suspect','SUSPECTS','Hall','BLDG','Apartments','Custody Date','Alert','LA','Off Campus',
'Field','O','Northwest','North','Alley','Ave', 'California', 'Los Angeles', 'Freeway','USC',
'University','America','on St','Eastside', 'New', 'BLDG', 'Northeast', 'Disposition', 'DPS Camera']


def remove_words(list1, remove_words):
    result = list(filter(lambda word: word not in remove_words, list1))
    return result

In [16]:
data['LOCATION'][130]

'Orchard Avenue and 28th'

In [17]:
#data['LOCATION']

chk_lst = ['Street','St','st','Boulevard','Blvd','Avenue','Field','Hall','Place','Ave','Parkway']

loc_lst=[]

rec_no=0
#for i in (data[data['LOCATION']=='Off-Campus, on the top floor of the USC Shrine Parking Structure at 631 Jefferson Blvd'])['LOCATION']:
for i in data['LOCATION']:
    rec_no=rec_no+1
    print("\n")
    print(rec_no)
    print(i)
    print("---------------------")
    
    loc=i
         
    #i = re.sub(r",", '', i)
    #i = re.sub(r";", '', i)
    
    i = i.translate(str.maketrans('', '', string.punctuation))
    i = i.strip()
    #print(i)

    
    str_lst=[]

    for sent in nltk.sent_tokenize(loc):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                #print(chunk.label(), ' '.join(c[0] for c in chunk))
                wrd = ' '.join(c[0] for c in chunk)
                for k in range(len(i.split())):
                    if((i.split()[k] in wrd)):
                        if(k < len(i.split())-1):
                            wrd1 = i.split()[k+1]
                            if(('Street' not in wrd) and ('St' not in wrd) and ('st.' not in wrd) 
                            and ('Boulevard' not in wrd) and ('Blvd' not in wrd) and ('Avenue' not in wrd) 
                            and ('Ave' not in wrd) and ('Field' not in wrd)
                            and ('Hall' not in wrd) and ('Place' not in wrd)):
                                if(wrd1 not in chk_lst):
                                    str_lst.append(wrd)  
                    
            
            
    if(i.count('Boulevard')>0):
        n=i.count('Boulevard')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Boulevard')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Boulevard')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                           
    if(i.count('Parkway')>0):
        n=i.count('Parkway')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Parkway')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Parkway')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
    if(i.count('Drive')>0):
        n=i.count('Drive')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Drive')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Drive')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
#     if(i.count('Boulevard.')>0):
#         n=i.count('Boulevard.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='Boulevard.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('Boulevard.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])
                        
                        
                        
    if(i.count('Blvd')>0):
        n=i.count('Blvd')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Blvd')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Blvd')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
#     if(i.count('Blvd.')>0):
#         n=i.count('Blvd.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='Blvd.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('Blvd.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])


                        
    if(i.count('Avenue')>0):
        n=i.count('Avenue')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Avenue')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Avenue')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
#     if(i.count('Avenue.')>0):
#         n=i.count('Avenue.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='Avenue.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('Avenue.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])
                        

    if(i.count('Ave')>0):
        n=i.count('Ave')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Ave')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Ave')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
    if(i.count('Field')>0):
        n=i.count('Field')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Field')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Field')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])


    if(i.count('Hall')>0):
        n=i.count('Hall')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Hall')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Hall')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])


    if(i.count('Place')>0):
        n=i.count('Place')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Place')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Place')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])


#     if(i.count('Place.')>0):
#         n=i.count('Place.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='Place.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('Place.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
    if(i.count('Street')>0):
        n=i.count('Street')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='Street')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('Street')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
#     if(i.count('Street.')>0):
#         n=i.count('Street.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='Street.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('Street.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])

    
    if(i.count('St')>0):
        n=i.count('St')
        #print("n = ",n)
        for k in range(len(i.split())):
            if((i.split()[k]=='St')):
                #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
                    s = " ".join(str_lst)
                    sc = s.count('St')
                    #print("str_lst = ", sc)
                    if(sc < n):
                        #print(i.split()[k-1]+" "+i.split()[k])
                        str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
#     if(i.count('St.')>0):
#         n=i.count('St.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='St.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('St.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])

                        
#     if(i.count('st.')>0):
#         n=i.count('st.')
#         #print("n = ",n)
#         for k in range(len(i.split())):
#             if((i.split()[k]=='st.')):
#                 #if(("th" in i.split()[k-1][-2:]) or ("nd" in i.split()[k-1][-2:]) or ("st" in i.split()[k-1][-2:]) or ("rd" in i.split()[k-1][-2:])):
#                     s = " ".join(str_lst)
#                     sc = s.count('st.')
#                     #print("str_lst = ", sc)
#                     if(sc < n):
#                         #print(i.split()[k-1]+" "+i.split()[k])
#                         str_lst.append(i.split()[k-1]+" "+i.split()[k])
    
    
    for x in range(len(str_lst)):
        if('Street' in str_lst[x]):
            str_lst[x] = str_lst[x].replace('Street', 'St')

        if('Blvd' in str_lst[x]):
            str_lst[x] = str_lst[x].replace('Blvd', 'Boulevard')

        if('Avenue' in str_lst[x]):
            str_lst[x] = str_lst[x].replace('Avenue', 'Ave')

        if('28TH St' in str_lst[x]):
            str_lst[x] = str_lst[x].replace('28TH St', '28th St')

    str_lst = remove_words(str_lst, remove_locs)    
    
    for x in str_lst:
        for y in str_lst:
            if((x in y) and (x!=y)):
                str_lst.remove(x)
                
    str_lst = list(set(str_lst)) 

    for j in str_lst:
        print(j)
        loc_lst.append(j)




1
Off campus at the Pacific Apartments at 2637 Severance Street
---------------------
Pacific Apartments
Severance St


2
Off campus on the sidewalk in front of the Domino’s Pizza at 2803 S. Figueroa Street
---------------------
Pizza
Figueroa St
Domino


3
Off-Campus, at the Metro train platform on Jefferson Boulevard and Flower Street
---------------------
Flower St
Jefferson Boulevard
Metro


4
Off-Campus, on the northeast corner of Catalina Street and Vermont Avenue, and east of Vermont Avenue on Jefferson Boulevard.
---------------------
Catalina St
Vermont Ave
Jefferson Boulevard


5
Off-Campus, at the Bank of America located at the southwest corner of 27th Street and Figueroa Street
---------------------
Bank
Figueroa St
27th St


6
Off-Campus, north of 32nd Street on the westside of Figueroa Street and on the southside of 32nd Street
---------------------
Figueroa St
32nd St


7
Off-Campus, at the Metro train platform on Exposition Boulevard
---------------------
Exposition B

Off campus at Amin’s Oil station at 2603 S. Normandie Ave.
---------------------
Normandie Ave


89
Off campus in the 2800 block of South Figueroa Street.
---------------------
Figueroa St


90
Off campus in the 1000 block of West 30th Street.
---------------------
30th St


91
Off-campus at Exposition Boulevard and Vermont Avenue on the East Metro platform.
---------------------
East Metro
Vermont Ave
Exposition Boulevard


92
Off-campus in the Shrine Auditorium parking structure at 3208 Royal Street.
---------------------
Shrine Auditorium
Royal St


93
Off campus in the 1200 block of West 25th Street
---------------------
25th St


94
Off campus at Jefferson Boulevard and Walton Avenue
---------------------
Jefferson Boulevard
Walton Ave


95
Off campus in the 1200 block of 27th Street
---------------------
27th St


96
Off campus in the 1100 block W. 29th Street.
---------------------
29th St


97
Off-campus in the 1100 block of Exposition Boulevard.
---------------------
Expositio

In [18]:
loc_lst

['Pacific Apartments',
 'Severance St',
 'Pizza',
 'Figueroa St',
 'Domino',
 'Flower St',
 'Jefferson Boulevard',
 'Metro',
 'Catalina St',
 'Vermont Ave',
 'Jefferson Boulevard',
 'Bank',
 'Figueroa St',
 '27th St',
 'Figueroa St',
 '32nd St',
 'Exposition Boulevard',
 'Metro',
 'Exposition Boulevard',
 'Metro',
 'Trousdale Parkway',
 'Vermont Ave',
 'Jefferson Boulevard',
 'University Gateway Apartments',
 'Figueroa St',
 'Figueroa St',
 'Exposition Boulevard',
 'Flower St',
 'Vermont Ave',
 'Exposition Boulevard',
 'Figueroa St',
 'Exposition Boulevard',
 'Jefferson Boulevard',
 'Jefferson Boulevard',
 'Trousdale Parkway',
 'Figueroa St',
 '31st St',
 '35th St',
 'Vermont Ave',
 'Figueroa St',
 'Exposition Boulevard',
 'Orchard Ave',
 '30th St',
 'Parkside Apartments',
 'Leavey Library',
 'Parkside International Residential College',
 'Marks Tower',
 'USC Bookstore',
 'University Village',
 'Hoover St',
 'University Ave',
 '37th Drive',
 'Vermont Ave',
 'Vermont Ave',
 'Orchard Ave

In [19]:
data['LOCATION'][92]

'Off-campus in the Shrine Auditorium parking structure at 3208 Royal Street.'

In [20]:
remove_locs=['Boulevard','Smart','Southeast','Similar','West','Jesse','Incident','Public Safety','DPS','Avenue',
'Custody','Aggravated','Sexual','Texts','Date','IRN','South','Homeless','Original Incident','New','SUSPECT',
'Black','Suspect','SUSPECTS','Hall','BLDG','Apartments','Custody Date','Alert','LA','Off Campus',
'Field','O','Northwest','North','Alley','Ave', 'California', 'Los Angeles', 'Freeway','USC',
'University','America','on St','Eastside', 'New', 'BLDG', 'Northeast', 'Disposition', 'DPS Camera']

def remove_words(list1, remove_words):
    result = list(filter(lambda word: word not in remove_words, list1))
    return result

print(len(loc_lst))
loc_lst = remove_words(loc_lst, remove_locs)
loc_lst
print(len(loc_lst))

315
315


In [21]:
loc_lst

['Pacific Apartments',
 'Severance St',
 'Pizza',
 'Figueroa St',
 'Domino',
 'Flower St',
 'Jefferson Boulevard',
 'Metro',
 'Catalina St',
 'Vermont Ave',
 'Jefferson Boulevard',
 'Bank',
 'Figueroa St',
 '27th St',
 'Figueroa St',
 '32nd St',
 'Exposition Boulevard',
 'Metro',
 'Exposition Boulevard',
 'Metro',
 'Trousdale Parkway',
 'Vermont Ave',
 'Jefferson Boulevard',
 'University Gateway Apartments',
 'Figueroa St',
 'Figueroa St',
 'Exposition Boulevard',
 'Flower St',
 'Vermont Ave',
 'Exposition Boulevard',
 'Figueroa St',
 'Exposition Boulevard',
 'Jefferson Boulevard',
 'Jefferson Boulevard',
 'Trousdale Parkway',
 'Figueroa St',
 '31st St',
 '35th St',
 'Vermont Ave',
 'Figueroa St',
 'Exposition Boulevard',
 'Orchard Ave',
 '30th St',
 'Parkside Apartments',
 'Leavey Library',
 'Parkside International Residential College',
 'Marks Tower',
 'USC Bookstore',
 'University Village',
 'Hoover St',
 'University Ave',
 '37th Drive',
 'Vermont Ave',
 'Vermont Ave',
 'Orchard Ave

In [22]:
my_dict = {i:loc_lst.count(i) for i in loc_lst}
len(my_dict)
my_dict

loc_df = pd.DataFrame(list(my_dict.items()),columns = ['LOCATION','COUNT'])
loc_df

loc_df = loc_df.sort_values(by='COUNT', ascending=False)
loc_df.head(50)
#loc_df.tail(50)

,LOCATION,COUNT
3,Figueroa St,42
6,Jefferson Boulevard,23
19,30th St,19
26,Hoover St,18
13,Exposition Boulevard,17
9,Vermont Ave,16
12,32nd St,11
11,27th St,10
48,28th St,9
39,29th St,9


In [23]:
loc_df = loc_df.reindex(['COUNT','LOCATION'], axis=1)
loc_df['LOCATION'].unique()

array(['Figueroa St', 'Jefferson Boulevard', '30th St', 'Hoover St',
       'Exposition Boulevard', 'Vermont Ave', '32nd St', '27th St',
       '28th St', '29th St', 'McClintock Ave', 'Orchard Ave', 'Royal St',
       'Flower St', 'Adams Boulevard', '37th Place', 'Ellendale Place',
       'Metro', 'Normandie Ave', '37th St', 'Trousdale Parkway',
       'Health Sciences Campus', '34th St', '35th St', 'Alcazar St',
       'Soto St', '23rd St', 'University Ave', '37th Drive', '36th Place',
       '31st St', 'Scarff St', 'Gardens Apartments', 'Hope St', 'Hill St',
       'South Hope', 'Bus', 'Adams Ave', 'Magnolia St',
       'Gateway Apartments', 'Ellendale Ave', 'Zonal Ave', 'Parkside',
       'Pacific Apartments', 'Troy Hall', 'Watt Way', 'Adams',
       'Health Science Campus', 'Playground St', 'CVS',
       'Brittingham Field', 'Washington Boulevard', 'Shrine',
       'Norfolk St', 'Valley Boulevard', 'HSC', 'Pablo St',
       'Portland Ave', '24th St', 'Gateway', 'South Figueroa',
  

In [24]:
loc_df

,COUNT,LOCATION
3,42,Figueroa St
6,23,Jefferson Boulevard
19,19,30th St
26,18,Hoover St
13,17,Exposition Boulevard
...,...,...
55,1,USC Village
56,1,La Sorbonne Apartments
58,1,East Metro
59,1,Shrine Auditorium


### MySQL

In [25]:
#!pip install PyMySQL
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

In [26]:
host = "studentnest.cwp11gjis9xd.us-east-1.rds.amazonaws.com"
port = 3306
dbname = "studentnest"
username = "admin"
password = "mysql1234"        

In [27]:
db = pymysql.connect(host='studentnest.cwp11gjis9xd.us-east-1.rds.amazonaws.com',
                             user='admin',
                             password='mysql1234',                             
                             db='studentnest',
                             port = 3306) 
print ("connect successful!!") 

connect successful!!


In [28]:
cursor = db.cursor()
cursor

In [29]:
cursor.execute("select version()")

1

In [30]:
data1 = cursor.fetchone()
data1

('8.0.27',)

### Queries

In [40]:
loc_df_lst = [tuple(x) for x in loc_df.to_records(index=False)]
loc_df_lst

[(42, 'Figueroa St'),
 (23, 'Jefferson Boulevard'),
 (19, '30th St'),
 (18, 'Hoover St'),
 (17, 'Exposition Boulevard'),
 (16, 'Vermont Ave'),
 (11, '32nd St'),
 (10, '27th St'),
 (9, '28th St'),
 (9, '29th St'),
 (8, 'McClintock Ave'),
 (6, 'Orchard Ave'),
 (6, 'Royal St'),
 (6, 'Flower St'),
 (5, 'Adams Boulevard'),
 (4, '37th Place'),
 (4, 'Ellendale Place'),
 (4, 'Metro'),
 (3, 'Normandie Ave'),
 (3, '37th St'),
 (3, 'Trousdale Parkway'),
 (3, 'Health Sciences Campus'),
 (3, '34th St'),
 (3, '35th St'),
 (2, 'Alcazar St'),
 (2, 'Soto St'),
 (2, '23rd St'),
 (2, 'University Ave'),
 (2, '37th Drive'),
 (2, '36th Place'),
 (2, '31st St'),
 (2, 'Scarff St'),
 (2, 'Gardens Apartments'),
 (1, 'Hope St'),
 (1, 'Hill St'),
 (1, 'South Hope'),
 (1, 'Bus'),
 (1, 'Adams Ave'),
 (1, 'Magnolia St'),
 (1, 'Gateway Apartments'),
 (1, 'Ellendale Ave'),
 (1, 'Zonal Ave'),
 (1, 'Parkside'),
 (1, 'Pacific Apartments'),
 (1, 'Troy Hall'),
 (1, 'Watt Way'),
 (1, 'Adams'),
 (1, 'Health Science Campus'),

In [41]:
# Initial insert to dps_alert_locations_frequency

# sql = "INSERT INTO dps_alert_locations_frequency(LOCATION,COUNT) values(%s, %s)"
# cursor.executemany(sql, loc_df_lst)
# db.commit()
# print(cursor.rowcount, "was inserted.")

In [42]:
sql = "ALTER TABLE dps_alert_locations_frequency DROP COLUMN ID"
cursor.execute(sql)
temp = cursor.fetchall()
list(temp)


sql = "ALTER TABLE housing_app_localerts DROP COLUMN ID"
cursor.execute(sql)
temp = cursor.fetchall()
list(temp)

[]

In [43]:
sql = "UPDATE dps_alert_locations_frequency SET COUNT = %s WHERE LOCATION = %s"
input_data = (loc_df['COUNT'], loc_df['LOCATION'])
cursor.executemany(sql, loc_df_lst)  #input_data
db.commit()
print(cursor.rowcount, "was updated in dps_alert_locations_frequency")


sql = "UPDATE housing_app_localerts SET COUNT = %s WHERE LOCATION = %s"
input_data = (loc_df['COUNT'], loc_df['LOCATION'])
cursor.executemany(sql, loc_df_lst)  #input_data
db.commit()
print(cursor.rowcount, "was updated in housing_app_localerts")

0 was updated in dps_alert_locations_frequency
0 was updated in housing_app_localerts


In [44]:
sql = '''select * from dps_alert_locations_frequency;'''
cursor.execute(sql)
d1 = cursor.fetchall()
list(d1)


sql = '''select * from housing_app_localerts;'''
cursor.execute(sql)
d1 = cursor.fetchall()
list(d1)

[('23rd St', 2),
 ('24th St', 1),
 ('25th St', 1),
 ('27th St', 10),
 ('28th St', 9),
 ('29th St', 9),
 ('30th St', 19),
 ('31st St', 2),
 ('32nd St', 11),
 ('34th St', 3),
 ('35th Place', 1),
 ('35th St', 3),
 ('36th Place', 2),
 ('36th St', 1),
 ('37th Drive', 2),
 ('37th Place', 4),
 ('37th St', 3),
 ('39th St', 1),
 ('Adams', 1),
 ('Adams Ave', 1),
 ('Adams Boulevard', 5),
 ('Alcazar St', 2),
 ('Bank', 1),
 ('Brittingham Field', 1),
 ('Bus', 1),
 ('Catalina St', 1),
 ('CVS', 1),
 ('Domino', 1),
 ('East Metro', 1),
 ('Ellendale Ave', 1),
 ('Ellendale Place', 4),
 ('Exposition Boulevard', 17),
 ('Felix Chevrolet', 1),
 ('Figueroa', 1),
 ('Figueroa St', 42),
 ('Flower', 1),
 ('Flower St', 6),
 ('Gardens Apartments', 2),
 ('Gateway', 1),
 ('Gateway Apartments', 1),
 ('Grand Ave', 1),
 ('Health Science Campus', 1),
 ('Health Sciences Campus', 3),
 ('Hill St', 1),
 ('Hoover Ave', 1),
 ('Hoover St', 18),
 ('Hope St', 1),
 ('HSC', 1),
 ('James Place', 1),
 ('Jefferson', 1),
 ('Jefferson Bo

In [45]:
d1_temp = []
for i in loc_df_lst:
    j, k = i[1], i[0]
    jk = (j,k)
    #print(jk)
    if jk not in d1:
        #print(i)  
        d1_temp.append(jk)

len(d1_temp)   
d1_temp

[]

In [46]:
# Insert to dps_alert_locations_frequency

sql = "INSERT INTO dps_alert_locations_frequency(LOCATION,COUNT) values(%s, %s)"
cursor.executemany(sql, d1_temp)
db.commit()
print(cursor.rowcount, "was inserted into dps_alert_locations_frequency")


# Insert to housing_app_localerts

sql = "INSERT INTO housing_app_localerts(LOCATION,COUNT) values(%s, %s)"
cursor.executemany(sql, d1_temp)
db.commit()
print(cursor.rowcount, "was inserted into housing_app_localerts")

98 was inserted into dps_alert_locations_frequency
98 was inserted into housing_app_localerts


In [47]:
sql = "ALTER TABLE dps_alert_locations_frequency ADD COLUMN ID INT AUTO_INCREMENT UNIQUE FIRST"
cursor.execute(sql)
temp = cursor.fetchall()
list(temp)


sql = "ALTER TABLE housing_app_localerts ADD COLUMN ID INT AUTO_INCREMENT UNIQUE FIRST"
cursor.execute(sql)
temp = cursor.fetchall()
list(temp)

[]

In [38]:
###########

In [72]:
data_main = pd.DataFrame() #data.copy()

# data_main['INCIDENT TYPE']=""
# data_main['INCIDENT DESCRIPTION']=""
# data_main['DATE & TIME OF OCCURRENCE']=""
# data_main['LOCATION']=""
# data_main['VEHICLE DESCRIPTION']=""
# data_main['SUSPECT DESCRIPTION']=""
# data_main

data_main['INCIDENT TYPE']=data['INCIDENT TYPE']
data_main['INCIDENT DESCRIPTION']=data['INCIDENT DESCRIPTION']
data_main['DATE & TIME OF OCCURRENCE']=data['DATE & TIME OF OCCURRENCE']
data_main['LOCATION']=data['LOCATION']
data_main['VEHICLE DESCRIPTION']=data['VEHICLE DESCRIPTION']
data_main['SUSPECT DESCRIPTION']=data['SUSPECT DESCRIPTION']
data_main['ALERT LINK']=data['ALERT LINK']

data_main

,INCIDENT TYPE,INCIDENT DESCRIPTION,DATE & TIME OF OCCURRENCE,LOCATION,VEHICLE DESCRIPTION,SUSPECT DESCRIPTION,ALERT LINK
INDEX,,,,,,,
1,Attempted Robbery,The victim exited the Metro platform at Jeffer...,Date & Time of Occurrence 04/04/2022 4:19 p.m.,"Off-Campus, at the Metro train platform on Jef...",Vehicle Description None,Suspect Description (provided by reporting party),https://dps.usc.edu/attempted-robbery-6/
2,Robbery Suspect in Custody,The juvenile male suspect approached a USC stu...,Date & Time of Occurrence 03/30/2022 11:51 a.m...,"Off-Campus, on the northeast corner of Catalin...",Vehicle Description None,Suspect Description (provided by reporting par...,https://dps.usc.edu/robbery-suspect-in-custody-4/
3,Robbery,"While at the ATM, the victim was approached by...",Date & Time of Occurrence 03/25/2022 3:41 a.m.,"Off-Campus, at the Bank of America located at ...",Vehicle Description Silver sedan with a sunroo...,Suspect Description (provided by reporting party),https://dps.usc.edu/robbery-82/
4,Sexual Battery/Fondling,INCIDENT DESCRIPTION A female student was wal...,03/09/2022 3:55 p.m. – 03/09/2022 9:20 p.m.,"Off-Campus, north of 32nd Street on the westsi...",Skateboard,(provided by reporting party),https://dps.usc.edu/sexual-battery-fondling-13/
5,Robbery,A USC student disembarked the Metro train on E...,02/20/2022 1:30 p.m.,"Off-Campus, at the Metro train platform on Exp...",None,(provided by reporting party),https://dps.usc.edu/robbery-81/
...,...,...,...,...,...,...,...
188,Robbery Arrest,"At approximately 9:30 PM, a student was skateb...",,,DESCRIPTION None. Suspect left the area on foot.,,https://dps.usc.edu/robbery-arrest-17/
189,Robbery/Kidnap Arrest,"While walking away from her residence, a stude...",,,,,https://dps.usc.edu/robbery-kidnap-arrest/
190,Attempt Robbery,A female student and her mother were walking w...,"January 24, 2018 at approximately 2:20 P.M.",37th Street west of Vermont Avenue (off campus),Dark colored scooter similar to an URB-E,"(provided by the reporting parties) Male, Bla...",https://dps.usc.edu/attempt-robbery-5/


In [73]:
data_lst = [tuple(x) for x in data_main.to_records(index=False)]
data_lst

[('Attempted Robbery',
  'The victim exited the Metro platform at Jefferson Boulevard and Flower Street when she was approached by the suspect. The suspect demanded her property and brandished a knife. The victim did not have any property to surrender, and the suspect left the area heading eastbound on Jefferson Boulevard. The victim was unharmed and is not affiliated with the university.',
  'Date & Time of Occurrence 04/04/2022 4:19 p.m.',
  'Off-Campus, at the Metro train platform on Jefferson Boulevard and Flower Street',
  'Vehicle Description None',
  'Suspect Description (provided by reporting party)',
  'https://dps.usc.edu/attempted-robbery-6/'),
 ('Robbery Suspect in Custody',
  'The juvenile male suspect approached a USC student at the northeast corner of Catalina Street and Vermont Avenue and demanded the student’s cell phone. The student did not surrender his property and fled from the location. A few minutes later, the same suspect approached a second USC student at Jeffe

In [74]:
# Initial insert to dps_alerts

# sql = "INSERT INTO dps_alerts(ALERT_LINK,INCIDENT_TYPE,INCIDENT_DESCRIPTION,DATE_TIME_OCCURRENCE,LOCATION,VEHICLE_DESCRIPTION,SUSPECT_DESCRIPTION) values(%s, %s, %s, %s, %s, %s, %s)"

# cursor.executemany(sql, data_lst)
# db.commit()
# print(cursor.rowcount, "was inserted.")

In [75]:
sql = "UPDATE dps_alerts SET INCIDENT_TYPE= %s, INCIDENT_DESCRIPTION= %s, DATE_TIME_OCCURRENCE= %s, LOCATION= %s, VEHICLE_DESCRIPTION= %s, SUSPECT_DESCRIPTION = %s WHERE ALERT_LINK = %s"
#input_data = (data_main['INCIDENT TYPE'], data_main['INCIDENT DESCRIPTION'], data_main['DATE & TIME OF OCCURRENCE'],data_main['LOCATION'], data_main['VEHICLE DESCRIPTION'], data_main['SUSPECT DESCRIPTION'], data_main['ALERT LINK'])
cursor.executemany(sql, data_lst)
db.commit()
print(cursor.rowcount, "was updated in dps_alerts")

0 was updated in dps_alerts


In [77]:
sql = '''select * from dps_alerts;'''
cursor.execute(sql)
d1 = cursor.fetchall()
list(d1)

[('https://dps.usc.edu/aggravated-assault-2/',
  'Aggravated Assault',
  'Incident:\xa0 Two males parked at the southwest corner of Hoover Street and 30th Street, exited their car and then walked a short distance west. The suspects then fired several gunshots at a man in front of them on 30th Street.',
  'Oct. 7, 2020 at 1 a.m.\nLocation\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Hoover Street and 30th Street (off-campus)',
  'Hoover Street and 30th Street (off-campus',
  'Suspect vehicle\xa0 Light gray 2013 Hyundai four-door sedan with moon roof.',
  ''),
 ('https://dps.usc.edu/aggravated-assault-3/',
  'Aggravated Assault',
  'Incident description While walking on the train platform, the victim (not affiliated with the university) was approached by the suspect who stabbed him in the chest with a sharp object.',
  '6:30 a.m. on Feb. 6, 2021',
  'Off-campus at Exposition Boulevard and Vermont Avenue on the East Metro platform.',
  'Suspect vehicle No vehicle seen.',
  'Suspect description Male, H

In [44]:
# j='https://dps.usc.edu/aggravated-assault-2/'

# for x in d1:
#     if j in x[0]:
#         print(j)

In [78]:
d1_lst=[]
for x in d1:
    y=x[0]
    d1_lst.append(y) 
d1_lst

['https://dps.usc.edu/aggravated-assault-2/',
 'https://dps.usc.edu/aggravated-assault-3/',
 'https://dps.usc.edu/aggravated-assault-4/',
 'https://dps.usc.edu/aggravated-assault-and-homicide/',
 'https://dps.usc.edu/aggravated-assault-homicide-suspect-deceased/',
 'https://dps.usc.edu/aggravated-assault-suspect-in-custody/',
 'https://dps.usc.edu/aggravated-assault-suspects-in-custody/',
 'https://dps.usc.edu/aggravated-assault/',
 'https://dps.usc.edu/armed-robbery-3/',
 'https://dps.usc.edu/armed-robbery-4/',
 'https://dps.usc.edu/assault-and-battery/',
 'https://dps.usc.edu/assault-with-a-deadly-weapon-2/',
 'https://dps.usc.edu/assault-with-a-deadly-weapon-3/',
 'https://dps.usc.edu/assault-with-a-deadly-weapon-4/',
 'https://dps.usc.edu/assault-with-a-vehicle/',
 'https://dps.usc.edu/attempt-robbery-4/',
 'https://dps.usc.edu/attempt-robbery-5/',
 'https://dps.usc.edu/attempt-robbery-6/',
 'https://dps.usc.edu/attempt-robbery-strong-arm/',
 'https://dps.usc.edu/attempt-street-rob

In [79]:
d1_temp = []
for i in data_lst:
    j0 = i[6]                 #(j1, j2, j3, j4, j5, j6 = i[0], i[1], i[2], i[3], i[4], i[5])
    j = (j0)               #(j1, j2, j3, j4, j5, j6)
    #print(j)
    
    if(j not in d1_lst):
        #print(j)  
        d1_temp.append(j)
        break

d1_temp   
#d1_temp

[]

In [81]:
data_lst

data_lst_2 = []
for i in data_lst:
    if(i[6] in d1_temp):
        #print(i)
        v = (i[6], i[0], i[1], i[2], i[3], i[4], i[5])
        data_lst_2.append(v)

data_lst_2

[]

In [83]:
# Insert to dps_alerts

sql = "INSERT INTO dps_alerts(ALERT_LINK,INCIDENT_TYPE,INCIDENT_DESCRIPTION,DATE_TIME_OCCURRENCE,LOCATION,VEHICLE_DESCRIPTION,SUSPECT_DESCRIPTION) values(%s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(sql, data_lst_2)
db.commit()
print(cursor.rowcount, "was inserted into dps_alerts")

192 was inserted into dps_alerts


In [50]:
# # Insert to dps_alerts_temp

# sql = "INSERT INTO dps_alerts_temp(ALERT_LINK,INCIDENT_TYPE,INCIDENT_DESCRIPTION,DATE_TIME_OCCURRENCE,LOCATION,VEHICLE_DESCRIPTION,SUSPECT_DESCRIPTION) values(%s, %s, %s, %s, %s, %s, %s)"

# cursor.executemany(sql, data_lst)
# db.commit()
# print(cursor.rowcount, "was inserted.")

### Views Update

In [84]:
# dps_alert_locations_frequency_ view update

sql = "CREATE OR REPLACE VIEW dps_alert_locations_frequency_vw AS Select * from dps_alert_locations_frequency;"

cursor.execute(sql)
db.commit()
#print(cursor.rowcount, "was inserted.")

In [85]:
# dps_alerts view update

sql = "CREATE OR REPLACE VIEW dps_alerts_vw AS Select * from dps_alerts;"

cursor.execute(sql)
db.commit()
#print(cursor.rowcount, "was inserted.")

In [86]:
# dps_alerts_vw
# dps_alert_locations_frequency_vw

cursor.close()